In [1055]:
# 매개인자로 크롤링하고 싶은 페이지 수
def source_crawler(page):
    # 필요한 라이브러리 임포트
    from bs4 import BeautifulSoup
    from urllib.request import urlopen, Request

    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys

    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.common.by import By

    from bs4 import BeautifulSoup
    import time
    
    page = page-1
    
    # selenium을 이용하여 웹페이지 소스 긁어오기
    chrome_driver = 'C:/Users/JIn/Documents/chromedriver' 

    # 크롤링할 url
    target_url = 'https://www.wine21.com/13_search/beer_list.html#List'

    # 상세보기를 들어가기 위한 목록리스트 source
    click_list=[]
    
    # 각 맥주의 상세보기 source
    soup = []
    try:
        driver = webdriver.Chrome(chrome_driver)
        driver.get(target_url)

        #목록 수 100개 설정 
        driver.find_element_by_xpath("//select/option[@value='100']").click()  

        list_source_len=len(driver.find_elements_by_xpath('//div[@class="tit"]/h4/a'))
        print('====================',1,"page :",list_source_len,"개",'====================')
                
    except:
        print('==========',1,"page 활성화 실패==========")
        pass
    
    finally:
        driver.close()
        
    for j in range(list_source_len):
        try:
            driver = webdriver.Chrome(chrome_driver)
            driver.get(target_url)

            #목록 수 100개 설정 
            driver.find_element_by_xpath("//select/option[@value='100']").click()    

            # 상세목록 클릭
            driver.find_elements_by_xpath('//div[@class="tit"]/h4/a')[j].click()

            # 페이지소스 저장
            src = driver.page_source
            soup.append(BeautifulSoup(src))
            print(1,"page :",j+1,"개 저장완료 ","현재개수 :",len(soup))
            
            driver.close()            
            
        except:
            print('==========',j+1,"page :",j+1,"번 저장실패 ",'==========')
        pass

                  
    for i in range(page):
        try:
            # 페이지 열기
            driver = webdriver.Chrome(chrome_driver)
            driver.get(target_url)
        
        
            #목록 수 100개 설정 
            driver.find_element_by_xpath("//select/option[@value='100']").click()    

            # 페이지 전환
            page = driver.find_elements_by_xpath('//div[@class="pagenate ver_web"]/a')
            page[i].click()

            list_source_len=len(driver.find_elements_by_xpath('//div[@class="tit"]/h4/a'))
            print('====================',i+2,"page :",list_source_len,"개",'====================')
            
            driver.close()
        except:
            print('==========',i+2,"page 활성화 실패==========")
            pass
                  
        for j in range(list_source_len):
            try:
                # 페이지 열기
                driver = webdriver.Chrome(chrome_driver)
                driver.get(target_url)

        
                # 목록 수 100개 설정 
                driver.find_element_by_xpath("//select/option[@value='100']").click()

                # 페이지 전환
                page = driver.find_elements_by_xpath('//div[@class="pagenate ver_web"]/a')
                page[i].click()

                # 상세목록 클릭
                driver.find_elements_by_xpath('//div[@class="tit"]/h4/a')[j].click()

                # 페이지소스 저장
                src = driver.page_source
                soup.append(BeautifulSoup(src))

                print(i+2,"page :",j+1,"개 저장완료 ","현재개수 : ",len(soup))
                driver.close()
          
            except:
                print('==========',i+2,"page :",j+1,"번 저장실패 ",'==========')
                pass

    return soup

In [1142]:
# 매개인자로 source_crawler함수의 list변수를 받음
def beer_mining(soup):
    import pandas as pd

    상품명_kor=[]
    상품명_eng=[]
    도수=[]
    원재료=[]
    홉종류=[]
    음용온도=[]
    IBU=[]
    SRM=[]
    정보=[]
    
    for i in range(len(soup)):
        #맥주 데이터수집
        source = soup[i].findAll("div",{"class":"column_detail2"})
        상품명_kor.append(source[0].findAll("h4",{"class":"name_ko"})[0].get_text())
        상품명_eng.append(source[0].findAll("div",{"class":"name_en"})[0].get_text())

        source02 = source[0].findAll("div",{"class":"wine_info"})[0] 
        도수.append(source02.findAll("dd")[3].get_text())
        원재료.append(source02.findAll("dd")[4].get_text())
        tmp=source02.findAll("dd")[5].get_text()
        홉종류.append(tmp.replace('\n',''))
        음용온도.append(source02.findAll("dd")[6].get_text())
        tmp=source02.findAll("dd")[7].get_text()
        IBU.append(tmp.replace('\xa0',''))
        SRM.append(source02.findAll("dd")[8].get_text()[0])

        #맥주정보 전처리 후 저장
        source03 =soup[i].findAll("div",{"class":"column_detail3"})
        try :
            info = source03[0].findAll('div',{'id':'MakerNote_wrap'})[0].get_text()
            thing = source03[0].findAll('span',{'class':'red'})[0].get_text()
            #info에서 thing이 들은 문자열 제거
            info=info.replace(thing,'')
        except:
            pass
        info=info.replace('\n','')    
        정보.append(info)
        
    return pd.DataFrame(
        data = {"상품명_kor":상품명_kor,"상품명_eng":상품명_eng,"도수":도수,"원재료":원재료,"홉종류":홉종류,"음용온도":음용온도,"IBU":IBU,"SRM":SRM,"정보":정보}
    )

In [1057]:
soup = source_crawler(12)

==================== 1 page : 100 개 ====================
1 page : 1 개 저장완료  현재개수 : 1
1 page : 2 개 저장완료  현재개수 : 2
1 page : 3 개 저장완료  현재개수 : 3
1 page : 4 개 저장완료  현재개수 : 4
1 page : 5 개 저장완료  현재개수 : 5
1 page : 6 개 저장완료  현재개수 : 6
1 page : 7 개 저장완료  현재개수 : 7
1 page : 8 개 저장완료  현재개수 : 8
1 page : 9 개 저장완료  현재개수 : 9
1 page : 10 개 저장완료  현재개수 : 10
1 page : 11 개 저장완료  현재개수 : 11
1 page : 12 개 저장완료  현재개수 : 12
1 page : 13 개 저장완료  현재개수 : 13
1 page : 14 개 저장완료  현재개수 : 14
1 page : 15 개 저장완료  현재개수 : 15
1 page : 16 개 저장완료  현재개수 : 16
1 page : 17 개 저장완료  현재개수 : 17
1 page : 18 개 저장완료  현재개수 : 18
1 page : 19 개 저장완료  현재개수 : 19
1 page : 20 개 저장완료  현재개수 : 20
1 page : 21 개 저장완료  현재개수 : 21
1 page : 22 개 저장완료  현재개수 : 22
1 page : 23 개 저장완료  현재개수 : 23
1 page : 24 개 저장완료  현재개수 : 24
1 page : 25 개 저장완료  현재개수 : 25
1 page : 26 개 저장완료  현재개수 : 26
1 page : 27 개 저장완료  현재개수 : 27
1 page : 28 개 저장완료  현재개수 : 28
1 page : 29 개 저장완료  현재개수 : 29
1 page : 30 개 저장완료  현재개수 : 30
1 page : 31 개 저장완료  현재개수 : 31
1 page : 32 개 저장완료  현재개수 : 32
1

3 page : 59 개 저장완료  현재개수 :  259
3 page : 60 개 저장완료  현재개수 :  260
3 page : 61 개 저장완료  현재개수 :  261
3 page : 62 개 저장완료  현재개수 :  262
3 page : 63 개 저장완료  현재개수 :  263
3 page : 64 개 저장완료  현재개수 :  264
3 page : 65 개 저장완료  현재개수 :  265
3 page : 66 개 저장완료  현재개수 :  266
3 page : 67 개 저장완료  현재개수 :  267
3 page : 68 개 저장완료  현재개수 :  268
3 page : 69 개 저장완료  현재개수 :  269
3 page : 70 개 저장완료  현재개수 :  270
3 page : 71 개 저장완료  현재개수 :  271
3 page : 72 개 저장완료  현재개수 :  272
3 page : 73 개 저장완료  현재개수 :  273
3 page : 74 개 저장완료  현재개수 :  274
3 page : 75 개 저장완료  현재개수 :  275
3 page : 76 개 저장완료  현재개수 :  276
3 page : 77 개 저장완료  현재개수 :  277
3 page : 78 개 저장완료  현재개수 :  278
3 page : 79 개 저장완료  현재개수 :  279
3 page : 80 개 저장완료  현재개수 :  280
3 page : 81 개 저장완료  현재개수 :  281
3 page : 82 개 저장완료  현재개수 :  282
3 page : 83 개 저장완료  현재개수 :  283
3 page : 84 개 저장완료  현재개수 :  284
3 page : 85 개 저장완료  현재개수 :  285
3 page : 86 개 저장완료  현재개수 :  286
3 page : 87 개 저장완료  현재개수 :  287
3 page : 88 개 저장완료  현재개수 :  288
3 page : 89 개 저장완료  현재개수 :  289
3 page :

6 page : 11 개 저장완료  현재개수 :  511
6 page : 12 개 저장완료  현재개수 :  512
6 page : 13 개 저장완료  현재개수 :  513
6 page : 14 개 저장완료  현재개수 :  514
6 page : 15 개 저장완료  현재개수 :  515
6 page : 16 개 저장완료  현재개수 :  516
6 page : 17 개 저장완료  현재개수 :  517
6 page : 18 개 저장완료  현재개수 :  518
6 page : 19 개 저장완료  현재개수 :  519
6 page : 20 개 저장완료  현재개수 :  520
6 page : 21 개 저장완료  현재개수 :  521
6 page : 22 개 저장완료  현재개수 :  522
6 page : 23 개 저장완료  현재개수 :  523
6 page : 24 개 저장완료  현재개수 :  524
6 page : 25 개 저장완료  현재개수 :  525
6 page : 26 개 저장완료  현재개수 :  526
6 page : 27 개 저장완료  현재개수 :  527
6 page : 28 개 저장완료  현재개수 :  528
6 page : 29 개 저장완료  현재개수 :  529
6 page : 30 개 저장완료  현재개수 :  530
6 page : 31 개 저장완료  현재개수 :  531
6 page : 32 개 저장완료  현재개수 :  532
6 page : 33 개 저장완료  현재개수 :  533
6 page : 34 개 저장완료  현재개수 :  534
6 page : 35 개 저장완료  현재개수 :  535
6 page : 36 개 저장완료  현재개수 :  536
6 page : 37 개 저장완료  현재개수 :  537
6 page : 38 개 저장완료  현재개수 :  538
6 page : 39 개 저장완료  현재개수 :  539
6 page : 40 개 저장완료  현재개수 :  540
6 page : 41 개 저장완료  현재개수 :  541
6 page :

8 page : 64 개 저장완료  현재개수 :  764
8 page : 65 개 저장완료  현재개수 :  765
8 page : 66 개 저장완료  현재개수 :  766
8 page : 67 개 저장완료  현재개수 :  767
8 page : 68 개 저장완료  현재개수 :  768
8 page : 69 개 저장완료  현재개수 :  769
8 page : 70 개 저장완료  현재개수 :  770
8 page : 71 개 저장완료  현재개수 :  771
8 page : 72 개 저장완료  현재개수 :  772
8 page : 73 개 저장완료  현재개수 :  773
8 page : 74 개 저장완료  현재개수 :  774
8 page : 75 개 저장완료  현재개수 :  775
8 page : 76 개 저장완료  현재개수 :  776
8 page : 77 개 저장완료  현재개수 :  777
8 page : 78 개 저장완료  현재개수 :  778
8 page : 79 개 저장완료  현재개수 :  779
8 page : 80 개 저장완료  현재개수 :  780
8 page : 81 개 저장완료  현재개수 :  781
8 page : 82 개 저장완료  현재개수 :  782
8 page : 83 개 저장완료  현재개수 :  783
8 page : 84 개 저장완료  현재개수 :  784
8 page : 85 개 저장완료  현재개수 :  785
8 page : 86 개 저장완료  현재개수 :  786
8 page : 87 개 저장완료  현재개수 :  787
8 page : 88 개 저장완료  현재개수 :  788
8 page : 89 개 저장완료  현재개수 :  789
8 page : 90 개 저장완료  현재개수 :  790
8 page : 91 개 저장완료  현재개수 :  791
8 page : 92 개 저장완료  현재개수 :  792
8 page : 93 개 저장완료  현재개수 :  793
8 page : 94 개 저장완료  현재개수 :  794
8 page :

11 page : 12 개 저장완료  현재개수 :  1012
11 page : 13 개 저장완료  현재개수 :  1013
11 page : 14 개 저장완료  현재개수 :  1014
11 page : 15 개 저장완료  현재개수 :  1015
11 page : 16 개 저장완료  현재개수 :  1016
11 page : 17 개 저장완료  현재개수 :  1017
11 page : 18 개 저장완료  현재개수 :  1018
11 page : 19 개 저장완료  현재개수 :  1019
11 page : 20 개 저장완료  현재개수 :  1020
11 page : 21 개 저장완료  현재개수 :  1021
11 page : 22 개 저장완료  현재개수 :  1022
11 page : 23 개 저장완료  현재개수 :  1023
11 page : 24 개 저장완료  현재개수 :  1024
11 page : 25 개 저장완료  현재개수 :  1025
11 page : 26 개 저장완료  현재개수 :  1026
11 page : 27 개 저장완료  현재개수 :  1027
11 page : 28 개 저장완료  현재개수 :  1028
11 page : 29 개 저장완료  현재개수 :  1029
11 page : 30 개 저장완료  현재개수 :  1030
11 page : 31 개 저장완료  현재개수 :  1031
11 page : 32 개 저장완료  현재개수 :  1032
11 page : 33 개 저장완료  현재개수 :  1033
11 page : 34 개 저장완료  현재개수 :  1034
11 page : 35 개 저장완료  현재개수 :  1035
11 page : 36 개 저장완료  현재개수 :  1036
11 page : 37 개 저장완료  현재개수 :  1037
11 page : 38 개 저장완료  현재개수 :  1038
11 page : 39 개 저장완료  현재개수 :  1039
11 page : 40 개 저장완료  현재개수 :  1040
11 page : 41 개

In [1060]:
len(soup)

1137

In [1144]:
df = beer_mining(soup)

In [1147]:
df.to_csv('beer_list(detail).csv', encoding='utf-8-sig')